In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from matplotlib import pyplot as plt
import numpy as np

from cassL import camb_interface as ci
from cassL import user_interface as ui

import camb
from camb import model, initialpower

import copy as cp

In [2]:
import importlib.util
import sys

spec = importlib.util.spec_from_file_location(
    "andreap", "../emulator/andreap.py")
andreap = importlib.util.module_from_spec(spec)
sys.modules["andreap"] = andreap
spec.loader.exec_module(andreap)

In [3]:
# Axe the 0.01 value since Ariel has not tested that one in the shared results
powernu3 = ci.load_benchmark(
    "../../power_nu3/Aletheia_powernu3_zorig_nu",
    ["0.0006", "0.0021", "0.0064", "0.0100"] 
)

In [ ]:
spectra = ci.boltzmann_battery(ci.OMNUH2_FLOATS, skips_omega=[],
                               skips_model=[8], k_points=1000,
                               skips_snapshot=[])

In [ ]:
Andrea_swarm = []
Ariel_swarm = []
k_axes = []

omnuh2_index = 0
for key in spectra:
    omnuh2 = key
 
    for modeli in range(len(spectra[key])):
        if modeli == 8:
            continue
            
        model = ci.cosm.iloc[modeli]
        model = ci.specify_neutrino_mass(model, omnuh2, 1)
        
        z_array = ci.parse_redshifts(modeli)
        
        for snapi in range(len(spectra[key][modeli])):
            z = z_array[snapi]
            
            Lukas_spec = spectra[key][modeli][snapi]
            Ariel_spec = powernu3[ci.OMNUH2_STRS[omnuh2_index]][modeli][snapi]
            
            # Possible bug? The code after this returned an error before this section did...
            # but shouldn't this code be executed first, always?
            if omnuh2_index == 0: # add the massless-neutrino case to our errors just once
                model_no = ci.specify_neutrino_mass(model, 0, 0)
                andrea_interpolator = andreap.cassL_to_andrea_cosmology(
                    model_no, hybrid=False)
                andrea_no_Pk = andrea_interpolator['mzero'].P(z, Lukas_spec["k"])
                Andrea_errors = (andrea_no_Pk - Lukas_spec["P_no"]) / andrea_no_Pk
                Andrea_swarm.append(Andrea_errors)
                Ariel_errors = (Ariel_spec["P_no"] - Lukas_spec["P_no"]) / Ariel_spec["P_no"]
                Ariel_swarm.append(Ariel_errors)
                k_axes.append(Lukas_spec["k"])
            
            andrea_interpolator = andreap.cassL_to_andrea_cosmology(
                model, hybrid=False)
            andrea_Pk = andrea_interpolator['mnu'].P(z, Lukas_spec["k"])
            Andrea_errors = (andrea_no_Pk - Lukas_spec["P_nu"]) / andrea_no_Pk
            Andrea_swarm.append(Andrea_errors)
            Ariel_errors = (Ariel_spec["P_nu"] - Lukas_spec["P_nu"]) / Ariel_spec["P_nu"]
            Ariel_swarm.append(Ariel_errors)
            k_axes.append(Lukas_spec["k"])
    
    omnuh2_index += 1

In [ ]:
for i in range(len(Ariel_swarm)):
    plt.plot(k_axes[i], 100 * Ariel_swarm[i], color="blue")

plt.ylabel("% error between CL and Sánchez codes")
plt.xlabel("Inverse scale $k$")
plt.title("Error Curves Using Sánchez's Spectra")

In [ ]:
for i in range(len(Andrea_swarm)):
    plt.plot(k_axes[i], Andrea_swarm[i], color="blue")

plt.ylabel("% error between CL and Pezzotta codes")
plt.xlabel("Inverse scale $k$")
plt.title("Error Curves Using Pezzotta's Spectra")